This notebook creates an mne compatible epoch file from the pre-processed data matrix generated in MATLAB

In [1]:
# If the structure of the matlab matrices is the same for all participants just edit this cell
channelabels_path = '/home/lea/Documents/obsidian_notes/masterADS/ads_thesis_RIFT/A0_kabir/preprocessing/channel_labels.mat'
eegdata_path = '/home/lea/Documents/obsidian_notes/masterADS/ads_thesis_RIFT/A0_kabir/converted/eeg_matrix.mat'
sampling_freq_hz = 2048
epoch_start_dist_to_cue = -1.0 #in secondes
participant_id = 0 # saving requires it to be an integer
endsave_path = '/home/lea/Documents/obsidian_notes/masterADS/ads_thesis_RIFT/my_code/epochs_from_matrix/'

In [2]:
import scipy.io as sio
import h5py
import numpy as np 
from mne import create_info, concatenate_raws, EpochsArray

In [3]:
# Get channels names from .mat
f = sio.loadmat(channelabels_path)
labels = f['channellabels_ADSselection']
labels[-1]

array([array(['EXG4'], dtype='<U4')], dtype=object)

In [4]:
# Remove external electrodes from labels
labels = labels[:-4]

In [5]:
# Load eeg data from .mat
f = h5py.File(eegdata_path)
eeg_data = np.array(f['data_eeg'])
eeg_data.shape # trials x timepoints x channels

(440, 6758, 37)

In [6]:
eeg_data = eeg_data.transpose((0,2,1))
trials, channels, timepoints = eeg_data.shape

In [7]:
# Define channel names
channel_names = [str(i[0][0]) for i in labels]

# Define sampling frequency 
sfreq = sampling_freq_hz

# Define time range such that cue onset = 0
tmin = epoch_start_dist_to_cue

# Create MNE info structure
info = create_info(channel_names, sfreq, ch_types='eeg')

# Enter participant ID
info['subject_info'] = {'id': participant_id}

# Built list of trial data
epochs_data = []  
for trial in range(trials):
    data = eeg_data[trial]
    epochs_data.append(data)

# Create EpochsArray object
epochs = EpochsArray(epochs_data, info, tmin=tmin)

Not setting metadata
440 matching events found
No baseline correction applied
0 projection items activated


In [8]:
epochs.set_montage('biosemi64')

Number of events,440
Events,1: 440
Time range,-1.000 – 2.299 sec
Baseline,off


In [9]:
# Save EpochsArray
epochs.save(f'{endsave_path}epochs_{participant_id}.fif', overwrite=True)

Overwriting existing file.


/tmp/ipykernel_22453/2167743718.py:2: RuntimeWarning: This filename (/home/lea/Documents/obsidian_notes/masterADS/ads_thesis_RIFT/my_code/epochs_from_matrix/epochs_0.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(f'{endsave_path}epochs_{participant_id}.fif', overwrite=True)
